<a href="https://colab.research.google.com/github/deepika-3/Deepa/blob/main/Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:

!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357274 sha256=ed4da93b6ac4d684d4151b3b612d86c34a3f814d64470031337a661de322b34c
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [13]:
import pandas as pd

# Example ratings DataFrame
data = {
    'userId': [1, 1, 1, 2, 2, 3, 3, 4, 4, 5],
    'movieId': [1, 2, 3, 1, 3, 2, 4, 1, 2, 3],
    'rating': [5, 3, 4, 4, 2, 5, 1, 2, 4, 5]
}
ratings = pd.DataFrame(data)


In [14]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split

# Load the data into Surprise format
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)

# Train the SVD model
model = SVD()
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)

# Evaluate the model
rmse = accuracy.rmse(predictions)
print(f'Root Mean Squared Error: {rmse:.2f}')


RMSE: 2.0392
Root Mean Squared Error: 2.04


In [15]:
# Function to get movie recommendations for a user
def get_recommendations(user_id, model, ratings, num_recommendations=5):
    # Get all movie IDs
    all_movie_ids = ratings['movieId'].unique()

    # Get the movies already rated by the user
    rated_movies = ratings[ratings['userId'] == user_id]['movieId'].tolist()

    # Predict ratings for all movies not rated by the user
    predictions = []
    for movie_id in all_movie_ids:
        if movie_id not in rated_movies:
            pred_rating = model.predict(user_id, movie_id).est
            predictions.append((movie_id, pred_rating))

    # Sort predictions by rating
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Return the top N movie recommendations
    return predictions[:num_recommendations]

# Example usage
user_id = 1
recommendations = get_recommendations(user_id, model, ratings)
print(f"Top recommendations for user {user_id}: {recommendations}")


Top recommendations for user 1: [(4, 3.814290949948592)]
